In [1]:
from __future__ import absolute_import
from __future__ import print_function
import os
import glob
import random
import numpy as np
from keras import optimizers
from keras.layers import LSTM
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16
from keras.layers.wrappers import TimeDistributed
from keras.applications.mobilenet import MobileNet
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.layers import Input, InputLayer
from keras.layers.core import Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.applications import imagenet_utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
videoFiles = glob.glob('../deep-dataset/numpys/*.npy')
mosFiles = [i for i in videoFiles if 'mos' in i]
videoFiles = [i for i in videoFiles if 'mos' not in i]

In [3]:
def myGenerator():
    while True:
        index_list = random.sample(range(1, 800), 2)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            #print(i, index_list)
            f = videoFiles[i]
            s = f[:-4]+'_mos.npy'
            a = np.load(f)
            b = np.load(s)
            alldata_x.append(a)
            alldata_y.append(b)
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        #print(alldata_x.shape, alldata_y.shape)
        yield alldata_x, alldata_y
# x = myGenerator()
# xtrain, ytrain = next(x)
# print('xtrain shape:',xtrain.shape)
# print('ytrain shape:',ytrain.shape)

In [4]:
print(len(videoFiles))
for i in videoFiles:
    print(np.load(i).shape)
    break

803
(200, 68, 120, 3)


In [5]:
height = 68
width = 120
input_shape=(200, height, width, 3)

In [15]:
def mySegNet(input_shape):
    base_model  = MobileNet(input_shape=(224,224,3), include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    cnn_model = Model(inputs=base_model.input, outputs=x)
    
    model = Sequential();
    #model.add(InputLayer(input_shape=input_shape))
    model.add(TimeDistributed(cnn_model, input_shape=input_shape))
    model.add(TimeDistributed(Flatten()))
    #model.add(cnn_model)
    #model.add(Flatten())
    
    model.add(LSTM(512, return_sequences=False))
    model.add(Dense(5, activation='softmax'))
    model.compile(optimizer = optimizers.Adam(lr=0.00001), loss='categorical_crossentropy')
    print(model.summary())
    return model 
#mySegNet(input_shape)

In [16]:
model = mySegNet(input_shape)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=20, epochs=50)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=20, epochs=50)
model.fit_generator(generator=myGenerator(),
                    use_multiprocessing=True,
                   steps_per_epoch=20, epochs=50)
# model.save('model1.h5')
# model.save_weights('model_weights1.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_7 (TimeDist (None, 200, 1024)         3228864   
_________________________________________________________________
time_distributed_8 (TimeDist (None, 200, 1024)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 512)               3147776   
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 2565      
Total params: 6,379,205
Trainable params: 6,357,317
Non-trainable params: 21,888
_________________________________________________________________
None
Epoch 1/50
20/20 [==============================] - 41s 2s/step - loss: 1.5657
Epoch 2/50
20/20 [==============================] - 27s 1s/step - loss: 1.6044
Epoch 3/50
20/20 [==============================] - 26s 1s/step - loss: 1.4879
Epoch 4/50
20/20 

20/20 [==============================] - 27s 1s/step - loss: 0.5444
Epoch 43/50
20/20 [==============================] - 27s 1s/step - loss: 0.7697
Epoch 44/50
20/20 [==============================] - 27s 1s/step - loss: 0.6372
Epoch 45/50
20/20 [==============================] - 27s 1s/step - loss: 0.6070
Epoch 46/50
20/20 [==============================] - 26s 1s/step - loss: 0.7094
Epoch 47/50
20/20 [==============================] - 27s 1s/step - loss: 0.6438
Epoch 48/50
20/20 [==============================] - 27s 1s/step - loss: 0.4917
Epoch 49/50
20/20 [==============================] - 27s 1s/step - loss: 0.5555
Epoch 50/50
20/20 [==============================] - 27s 1s/step - loss: 0.5882
Epoch 1/50
20/20 [==============================] - 27s 1s/step - loss: 0.5677
Epoch 2/50
20/20 [==============================] - 27s 1s/step - loss: 0.5648
Epoch 3/50
20/20 [==============================] - 27s 1s/step - loss: 0.5870
Epoch 4/50
20/20 [==============================] - 26s

In [25]:
x = myGenerator()
xtest, ytest = next(x)
pred = model.predict(xtest)
print(ytest, pred)

[[0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]] [[0.00143433 0.10271627 0.8425288  0.05015949 0.0031611 ]
 [0.7187001  0.00611032 0.26545057 0.00120025 0.00853875]]
